In [4]:
# No writes assumed
LOCAL_CACHE_SIZE = 4
GLOBAL_MEMORY_SIZE = 10000000
NUM_ELEMS_TO_ACCESS = 5
NUM_RUNS = 10

import time

class Cache:
  def __init__(self, cache_size = 4, global_memory_size = 8):
    self.cache_size = cache_size
    self.global_memory_size = global_memory_size
    self.cache = [{}, {}]
    self.global_memory = {}
    self.time_access = [{}, {}]
    for i in range(0, global_memory_size):
      self.global_memory[i] = i

  def get(self, cpu_no, address):
    if address in self.cache[cpu_no]:
      return self.cache[cpu_no][address]

    value = self.global_memory[address]
    if len(self.cache[cpu_no]) == self.cache_size:
      # TODO: Least recently used item to be replaced
      min_time = 0
      dic_addr = -1
      # Find in diciontary self.time_access[cpu_no] the address with lowest time and kick that out
      for dic_addr, dic_time in self.time_access[cpu_no].items():
        if  dic_time < min_time:
          min_time = dic_time
          min_addr = dic_addr
        # TODO: Find minimum
      del self.time_access[cpu_no][dic_addr]
      del self.cache[cpu_no][dic_addr]
      # TODO: delete an item from dictionary
      # HINT: Use delete to delete an entry in the dictionary.
      #self.cache[cpu_no].popitem()
    self.cache[cpu_no][address] = value
    # TODO: Add time stamp to track
    self.time_access[cpu_no][address] = time.time()
    return value

  def set(self, cpu_no, address, value):
    pass


def run_test(id_test):
  c = Cache(LOCAL_CACHE_SIZE, GLOBAL_MEMORY_SIZE)

  NUM_ITER = 1000000
  start = time.time()
  perc = NUM_ELEMS_TO_ACCESS
  for i in range(0, NUM_ITER):
    val = c.get(0, i % perc)
    if (val != (i% perc)):
      print("Error", val)

  end = time.time()
  dif_time = end - start
  print("Final time {0} for test {1}".format(dif_time, id_test))
  return dif_time

# Running tests NUM_RUNS times
time_tests = [0] * NUM_RUNS
for i in range(0, NUM_RUNS):
  time_tests[i] = run_test(i)

# Computing average of runtime
sum = 0.0
for time_test in time_tests:
  sum += time_test
print("Average time is", sum / len(time_tests))

Final time 0.23387432098388672 for test 0
Final time 0.23299288749694824 for test 1
Final time 0.23276400566101074 for test 2
Final time 0.23097610473632812 for test 3
Final time 0.23002076148986816 for test 4
Final time 0.23306870460510254 for test 5
Final time 0.23174405097961426 for test 6
Final time 0.2284071445465088 for test 7
Final time 0.22924280166625977 for test 8
Final time 0.2291109561920166 for test 9
Average time is 0.23122017383575438


In [1]:
# Parallel sum computation
# Parallel product computation
from multiprocessing import Process
from multiprocessing import Pool
import os
import time

NUM_PROC = 5
LEN_ARRAY = 40000000

def init_worker(array):
  global shared_array
  shared_array = array

def info(title, retVal):
    print(title)

def sum_array(array, startIdx, endIdx):
  sum = 0.0
  lenArray = len(array)
  for idx in range(startIdx, min(endIdx, lenArray)):
    sum += array[idx]
  return sum


def sum_array_parallel_function(numProc, idxArr):
  start = time.time()
  # Get the chunk size
  chunkSize = (len(shared_array) + numProc - 1) // numProc
  startIdx = chunkSize * idxArr
  endIdx = chunkSize * (idxArr + 1)
  #print("Idx", startIdx, endIdx)

  # Compute  the sum of subarray starting at startIdx position all the way up to endIdx
  sum = sum_array(shared_array, startIdx, endIdx)
  end = time.time()
  print("Process {0} runtime: {1}".format(idxArr, end - start))
  return sum


def sum_array_parallel(array, NUM_PROC):
  retVals = [0] * NUM_PROC
  procs = [0] * NUM_PROC

  listArgs = []
  for i in range(NUM_PROC):
    listArgs.append((NUM_PROC, i))

  start = time.time()
  pool = Pool(initializer=init_worker, initargs=(array,), processes=NUM_PROC)
  data = pool.starmap(sum_array_parallel_function, listArgs)
  pool.close()

  print(data)
  sum = sum_array(data, 0, NUM_PROC)
  print("Final sum:", sum)
  end = time.time()
  print("Final time {0} for number of processes {1}".format(end - start, NUM_PROC))


array = [i for i in range(0, LEN_ARRAY)]


sum_array_parallel(array, NUM_PROC)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'init_worker' on <module '__main__' (built-in)>


KeyboardInterrupt: 